In [4]:
import pandas as pd
from marketscraper import pricesearch
import os

In [5]:
os.chdir("C:/Users/claud/Documents/GitHub/economia/mapa_gro")

In [6]:
# url="https://raw.githubusercontent.com/labdatos-se/economia/main/mapa_gro/precios_pacic_gro30oct2023.csv"
# df=pd.read_csv(url,encoding='utf-8')
df=pd.read_csv("precios_pacic_gro30oct2023.csv",encoding='utf-8')

#Eliminar comas en precio
df['precio']=df['precio'].str.replace(',','')
#Precio a float
df['precio']=df['precio'].astype(float)
#Eliminar productos que tienen na
df["producto"]=df["producto"].fillna("NA")
df=df[df['producto']!='NA']
df

,producto,marca,precio,tienda,fecha_consulta,busqueda
0,tortilla taquera de maíz blanco kg,tortillería castro,26.500,super chedraui ometepec,2023-10-30 03:27:08.292618,tortilla de maíz
1,tiritas de maíz natural 1 pieza,selecto brand,39.000,super chedraui ometepec,2023-10-30 03:27:08.292618,tortilla de maíz
2,tortilla de maíz amarillo kg,tortillería castro,0.008,super chedraui ometepec,2023-10-30 03:27:08.292618,tortilla de maíz
3,tortilla de maíz por kilo,NaN,13.000,Bodega Aurrera Chilpancingo Norte,2023-10-30 03:28:18.900876,tortilla de maíz
4,harina de maíz maseca nixtamalizado 1 kg...,NaN,23.000,Bodega Aurrera Chilpancingo Norte,2023-10-30 03:28:18.900876,tortilla de maíz
...,...,...,...,...,...,...
39168,treda 20 tabletas,NaN,169.000,Bodega Aurrera Chilpancingo,2023-10-30 13:10:11.703700,antidiarreicos
39169,antiácido pepto bismol suspensión sabor origi...,NaN,102.000,Bodega Aurrera Chilpancingo,2023-10-30 13:10:11.703700,antidiarreicos
39170,antiácido pepto bismol suspensión sabor origi...,NaN,174.000,Bodega Aurrera Chilpancingo,2023-10-30 13:10:11.703700,antidiarreicos
39171,antiácido pepto bismol plus suspensión sabor o...,NaN,122.000,Bodega Aurrera Chilpancingo,2023-10-30 13:10:11.703700,antidiarreicos


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 39161 entries, 0 to 39172
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   producto        39161 non-null  object 
 1   marca           5691 non-null   object 
 2   precio          39157 non-null  float64
 3   tienda          39161 non-null  object 
 4   fecha_consulta  39161 non-null  object 
 5   busqueda        39161 non-null  object 
dtypes: float64(1), object(5)
memory usage: 2.1+ MB


In [7]:
df["busqueda"].unique()

array(['tortilla de maíz', 'aceite', 'arroz', 'atún en lata',
       'chuleta de puerco', 'cebolla', 'chile jalapeño', 'bistec de res',
       'frijol', 'huevo', 'jitomate', 'limón', 'manzana', 'naranja',
       'papa', 'pollo', 'sardina en lata', 'zanahoria', 'azúcar',
       'jabón de tocador', 'leche', 'pan de caja', 'papel higiénico',
       'pasta para sopa', 'plátano', 'lechuga', 'espinaca', 'lenteja',
       'jamón', 'mojarra tilapia', 'puré de tomate', 'servilleta',
       'toalla femenina', 'agua embotellada', 'pañales',
       'fórmulas lácteas', 'refresco', 'suero', 'antigripales',
       'antidiarreicos'], dtype=object)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48397 entries, 0 to 48402
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   producto        48397 non-null  object 
 1   precio          48397 non-null  float64
 2   tienda          48397 non-null  object 
 3   fecha_consulta  48397 non-null  object 
 4   busqueda        48397 non-null  object 
 5   marca           10725 non-null  object 
dtypes: float64(1), object(5)
memory usage: 2.6+ MB


In [45]:
def limpiaprod(df):
    #Eliminar acentos
    df["producto"]=df["producto"].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
    #Tortilla de maíz

    mask_tortilla = df["producto"].str.contains("tortilla de maíz", na=False)
    mask_not_masa = ~df["producto"].str.contains("tiritas", na=False)
    
    tortilla = df[mask_tortilla & mask_not_masa]

#    # Promedio y desviación estándar
    mean_price = tortilla["precio"].mean()
    std_price = tortilla["precio"].std() 

    # Filtrar aquellos que estén a 1 desviación estándar del promedio
    tortilla = tortilla[(tortilla["precio"] >= mean_price - 1 * std_price) & 
                        (tortilla["precio"] <= mean_price + 1 * std_price)]

    ##Aceite
    # mask_aceite = df["producto"].str.contains("aceite", na=False)
    # #Unidades 800ml, 946ml, 1 l en producto
    # mask_unidades = df["producto"].str.contains("700ml|800ml|875ml|946ml|1 l", na=False)
    # aceite=df[mask_aceite & mask_unidades]
    # #Eliminar si dice "transmisión" o "motocicletas"
    # aceite=aceite[~aceite["producto"].str.contains("trasmisión|motocicletas", na=False)] 
    # mean_price = aceite["precio"].mean()
    # std_price = aceite["precio"].std()

    # aceite = aceite[(aceite["precio"] >= mean_price - 1 * std_price) & 
    #                     (aceite["precio"] <= mean_price + 1 * std_price)]

   
   
   #tabla concatenada
    #finaldf=pd.concat([tortilla,aceite], axis=0)
    finaldf=tortilla
    
    finaldf.reset_index(drop=True, inplace=True)
    print(len(finaldf))
    
    return mean_price, std_price


In [80]:
def filtro_extremo(df, num_std=1):
    mean_value = df["precio"].mean()
    std_value = df["precio"].std()

    return df[(df["precio"] >= mean_value - num_std * std_value) & 
              (df["precio"] <= mean_value + num_std * std_value)]

def limpiaprod(df):
    #Eliminar acentos
    df["producto"]=df["producto"].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

    # Filtro inicial
    tortilla = df.query("`producto`.str.contains('tortilla de maiz') and not `producto`.str.contains('tiritas')")
    # Filtro del precio con una desviación estándar
    tortilla = filtro_extremo(tortilla, 1)

    #Aceite
    aceite=df.query("`producto`.str.contains('aceite') and `producto`.str.contains('700ml|800ml|875ml|946ml|1 l') and not `producto`.str.contains('trasmision|motocicletas|oliva')")
    # Filtro del precio con una desviación estándar
    aceite = filtro_extremo(aceite, 2)

    #tabla concatenada
    finaldf=pd.concat([aceite], axis=0)
    finaldf.reset_index(drop=True, inplace=True)
    return finaldf


In [81]:
x=limpiaprod(df)
#Dejar solo aceite
x


,producto,marca,precio,tienda,fecha_consulta,busqueda
0,aceite chedraui mixto 800ml,chedraui,35.0,super chedraui ometepec,2023-10-30 03:42:03.631345,aceite
1,aceite patrona mixto 1 l,patrona,51.5,super chedraui ometepec,2023-10-30 03:42:03.631345,aceite
2,aceite gran tradicion soya-canola 800ml,gran tradición,36.0,super chedraui ometepec,2023-10-30 03:42:03.631345,aceite
3,aceite vegetal nutrioli antigoteo puro de soya...,nutrioli,45.0,super chedraui ometepec,2023-10-30 03:42:03.631345,aceite
4,aceite de soya imperial 875ml,imperial,45.9,super chedraui ometepec,2023-10-30 03:42:03.631345,aceite
...,...,...,...,...,...,...
64,aceite comestible selecto brand puro de soya 8...,selecto brand,39.9,super chedraui teloloapan,2023-10-30 03:55:07.029756,aceite
65,aceite vegetal 123 1 l,NaN,51.0,Bodega Aurrera Zihuatanejo,2023-10-30 03:55:36.428284,aceite
66,aceite vegetal patrona 1 l,NaN,45.0,Bodega Aurrera Zihuatanejo,2023-10-30 03:55:36.428284,aceite
67,aceite vegetal 123 1 l,NaN,51.0,Bodega Aurrera Chilpancingo,2023-10-30 03:56:03.621371,aceite


In [82]:
#Obtener máximo y mínimo de precios
print(x["precio"].max())
print(x["precio"].min())

51.5
33.9
